<a href="https://colab.research.google.com/github/Nick088Official/GFPGAN-Fix/blob/master/GFPGAN_fix_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFPGAN Inference Demo  (Restore faces of images/videos)

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2101.04061)
[![GitHub Stars](https://img.shields.io/github/stars/TencentARC/GFPGAN?style=social)](https://github.com/TencentARC/GFPGAN)
[![download](https://img.shields.io/github/downloads/TencentARC/GFPGAN/total.svg)](https://github.com/TencentARC/GFPGAN/releases)

## GFPGAN - Towards Real-World Blind Face Restoration with Generative Facial Prior

GFPGAN is a blind face restoration algorithm towards real-world face images. <br>
It leverages the generative face prior in a pre-trained GAN (*e.g.*, StyleGAN2) to restore realistic faces while precerving fidelity. <br>

**Limitations**: GFPGAN could not handle all the low-quality faces in the real world. Therefore, it may fail on your own cases.

###Enjoy! :-)

<img src="https://xinntao.github.io/projects/GFPGAN_src/gfpgan_teaser.jpg" width="800">


In [ ]:
#@title Installation
#@markdown Before start, make sure that you choose
#@markdown * Runtime Type = Python 3
#@markdown * Hardware Accelerator = GPU

#@markdown in the **Runtime** menu -> **Change runtime type**

#@markdown By running this, we clone the repository, set up the envrironment, and download the pre-trained model which we use 1.2 , 1.3 , 1.4 , RestoreFormer , CodeFormer.

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN-Fix
!git clone https://github.com/Nick088Official/GFPGAN-Fix.git
%cd GFPGAN-Fix

import torch
from IPython.display import clear_output
import numpy as np
import glob
from os.path import isfile, join
import os
from google.colab import files
import shutil

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Set up the environment
# We install and use new-BasicSR for both training and inference
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
!pip uninstall -y basicsr # have to uninstall it as it gives error to GFPGAN but still gets installed from other package
!pip install -r requirements.txt # which installs other packages such as new-basicsr

if device == "cuda":
  !BASICSR_EXT=True pip install new-basicsr # for installing cuda extensions for v1 model for colorization

# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/CodeFormer.pth -P experiments/pretrained_models
clear_output()
print("Installed GFPGAN, its requirements and the models.")

Installed GFPGAN, its requirements and the models.


## Inference Images
Usage: `python inference_gfpgan.py -i inputs/upload -o results [options]...`

Options:
```
  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3 | 1.4. | RestoreFormer
  -s upscale           The final upsampling scale of the image. Default: 2
  -bg_upsampler        background upsampler. Default: realesrgan
  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
  -suffix              Suffix of the restored faces
  -only_center_face    Only restore the center face
  -aligned             Input are aligned faces
  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto
```

In [ ]:
#@title Upload Input Image

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload input
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'moved {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
#@title Inference

!rm -rf results

options = "-v 1.3 -s 2 --bg_upsampler realesrgan" #@param {type:"string"}

!python inference_gfpgan.py -i inputs/upload -o results {options}

!ls results/cmp

In [ ]:
#@title Visualize Input vs Output Faces
# We first visualize the cropped faces
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'results/cropped_faces'
result_folder = 'results/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)


In [ ]:
#@title View the whole Input vs Output Image
# We then visualize the whole image
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'inputs/upload'
result_folder = 'results/restored_imgs'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

In [ ]:
#@title Download results
#@markdown It will download you a .zip, unzip it however you want
#@markdown - The 'cmp' folder contains a comparison between the Input vs Output faces,
#@markdown - The 'cropped_faces' folder contains only the cropped Input Face
#@markdown - The 'restored_faces' folder contains only the Output Face (Restored one)
#@markdown - The 'restored_imgs' folder contains the whole Output Image restored with also the Background

# download the result
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")

## Inference Videos
Usage: `python inference_gfpgan.py -i inputs/upload -o results [options]...`

Options:
```
  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3 | 1.4. | RestoreFormer
  -s upscale           The final upsampling scale of the image. Default: 2
  -bg_upsampler        background upsampler. Default: realesrgan
  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
  -suffix              Suffix of the restored faces
  -only_center_face    Only restore the center face
  -aligned             Input are aligned faces
  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto
```

In [ ]:
#@title Upload Input Video
import os
from google.colab import files
import shutil
import cv2
from google.colab.patches import cv2_imshow

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")

else :
  os.mkdir(result_folder)


upload_folder = 'videos'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload input
uploaded = files.upload()
for filename in uploaded.keys():
  cap = cv2.VideoCapture(filename)
  fps = cap.get(cv2.CAP_PROP_FPS)
  dst_path = os.path.join(upload_folder, filename)
  print(f'moved {filename} of {fps} fps to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
#@title Inference
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output

# assign directory
directory = 'videos' #PATH_WITH_INPUT_VIDEOS
zee = 0

#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


for filename in os.listdir(directory):

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):


      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path

      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('upload'):
              os.makedirs('upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0


      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = 'upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
              print(currentframe)
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f))

              break

        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()

      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x

      #in the line below '2' stands for upscaling by factor of 2
      options = "-v 1.3 -s 2 --bg_upsampler realesrgan" #@param {type:"string"}

      !python inference_gfpgan.py -i upload -o results {options}

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder):
          os.remove(os.path.join(upload_folder, f))


      #convert super res frames to .avi
      pathIn = 'results/restored_imgs/'

      zee = zee+1
      fName = "video"+str(zee)
      filenameVid = f"{fName}.avi"

      pathOut = "results_videos/"+filenameVid

      fps = 25.0 #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir('results/restored_imgs'):
          os.remove(os.path.join('results/restored_imgs', f))

      #convert .avi to .mp4
      src = 'results_videos/'
      dst = 'results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
              except:
                  print("An exception occurred")

      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder):
          os.remove(os.path.join(video_result_folder, f))

In [ ]:
#@title Display Input vs Output Video
import cv2
import numpy as np

# Load input and output videos
input_video = cv2.VideoCapture("/content/GFPGAN-Fix/upload/")
output_video = cv2.VideoCapture("/content/GFPGAN-Fix/results_mp4_videos/")

# Get video dimensions
input_width, input_height = int(input_video.get(3)), int(input_video.get(4))
output_width, output_height = int(output_video.get(3)), int(output_video.get(4))

# Create a blank canvas to display both videos side by side
canvas_width = input_width + output_width
canvas_height = max(input_height, output_height)
canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

# Read frames from input and output videos
while True:
    input_frame_exists, input_frame = input_video.read()
    output_frame_exists, output_frame = output_video.read()

    if not input_frame_exists or not output_frame_exists:
        break

    # Resize frames to fit the canvas
    input_frame = cv2.resize(input_frame, (input_width, input_height))
    output_frame = cv2.resize(output_frame, (output_width, output_height))

    # Place input frame on the left and output frame on the right
    canvas[:, :input_width] = input_frame
    canvas[:, input_width:] = output_frame

    # Display the canvas
    cv2.imshow("Input and Output Videos", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close the window
input_video.release()
output_video.release()
cv2.destroyAllWindows()
